In [8]:
## RPC 服务
##%overwritefile
##%file:../src/_rpcservice.py
##%noruncode
     
    def start_srvmode(self,magics,rpcsrvobj,rpcurl=None):
        ## 使用线程启动
        if self._rpcsrv_thread != None:
            return
        self._rpcsrv_thread = Thread(target=MyMagics.rpc_srv, args=(self,magics,rpcsrvobj,rpcurl))
        self._rpcsrv_thread.daemon = True
        self._logln("srvmode start...")
        self._rpcsrv_thread.start()
    def rpc_srv(kobj,magics,rpcsrvobj,rpcurl=None):
        rurl=rpcurl
        # kobj.__rpcsrv = None
        if rpcurl==None:
            rurl=kobj.get_magicsSvalue(magics,'srvmode')
        if rpcsrvobj!=None and rurl!=None and len(rurl)>0:
            try:
                kobj.__rpcsrv = zerorpc.Server(rpcsrvobj)
                kobj.__rpcsrv._context.setsockopt(socket.SO_REUSEADDR, 1)
                kobj.__rpcsrv.bind(rurl)
                kobj.__rpcsrv.run()
            except Exception as e:
                kobj._logln("start_srvmode err:"+str(e),3)
                if kobj.__rpcsrv!=None :kobj.__rpcsrv.close()
        else:
            kobj._logln("srvmode err！",3)
        return 
    def stop_srvmode(self):
        if self.__rpcsrv!=None :
            try:
                rurl=self.get_magicsSvalue(self.first_magics,'srvmode')
                self.__rpcsrv.disconnect(rurl)
                self.__rpcsrv.stop()
                self.__rpcsrv.close()
                self.__rpcsrv._context.destroy()
                # self._put2stdin_queue = None
            except Exception as e:
                pass
        # self._rpcsrv_thread.join()

[MyPythonKernel211735] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_rpcservice.py created successfully


In [6]:
## RPC 客户端
##%overwritefile
##%file:../src/_getrpccli.py
##%noruncode
    def get_rpcsrvobj(self,magics,rpcurl=None):
        rpcsrvobj=None
        if rpcurl==None:
            rpcurl=self.get_magicsSvalue(magics,'srvurl')
        if rpcurl!=None and len(rpcurl)>0:
            try:
                ## self._logln('rpcsrvobj.connect '+rpcurl)
                rpcsrvobj = zerorpc.Client()
                rpcsrvobj.connect(rpcurl)
            except Exception as e:
                ## self._logln("rpccli err:"+str(e),3)
                if rpcsrvobj!=None :rpcsrvobj.close()
                rpcsrvobj=None
        return rpcsrvobj
    def send_stdincmd(self,magics,rpcurl,cmdstr):
        ## 发送命令后关闭rpc连接
        if rpcurl==None or len(rpcurl.strip())<3:
            rpcurl=self.get_magicsSvalue(magics,'srvurl')
        if rpcurl!=None and len(rpcurl.strip())>0:
            try:
                ## self._logln('get_rpcsrvobj '+rpcurl)
                rpcsrvobj=self.get_rpcsrvobj(magics,rpcurl)
                if rpcsrvobj!=None:
                    ## self._logln('rpcsrvobj.stdincmd '+cmdstr)
                    rpcsrvobj.stdincmd(cmdstr)
                    rpcsrvobj.close()
            except Exception as e:
                pass
    def send_cmd(self,magics,rpcurl,cmdstr):
        ## 发送命令后关闭rpc连接
        if rpcurl==None or len(rpcurl.strip())<3:
            rpcurl=self.get_magicsSvalue(magics,'srvurl')
        if rpcurl!=None and len(rpcurl.strip())>0:
            try:
                ## self._logln('send_cmd '+rpcurl)
                rpcsrvobj=self.get_rpcsrvobj(magics,rpcurl)
                if rpcsrvobj!=None:
                    ## self._logln('rpcsrvobj.cmd '+cmdstr)
                    rpcsrvobj.cmd(cmdstr)
                    rpcsrvobj.close()
            except Exception as e:
                pass
    def exec_rpccmd(self,magics,rpcurl=None,func=None,*args,**kwargs):
        if self.rpcsrvobj==None and rpcurl!=None and len(rpcurl)>0:
            self.rpcsrvobj=self.get_rpccli(magics,rpcurl=rpcurl)
        if self.rpcsrvobj!=None and func!=None:
            ret=func(*args,**kwargs)
            return ret
        return None


[MyPythonKernel211735] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_getrpccli.py created successfully


In [7]:
## RPC 服务类
##%overwritefile
##%file:../src/_RPCClass.py
##%noruncode
class RPCsrv(object):

    def __init__(self,kobj,magics):
        self.kobj=kobj
        self.magics=magics
    def output(self, contents):
        self.kobj._logln(contents)
        return
    def stdincmd(self,cmdstr,outencode='UTF-8'):
        pass
        ## self.kobj._logln("stdincmd received:"+cmdstr)
        ## 放入stdin队列里
        if self.kobj._put2stdin_queue.full(): return ''
        self.kobj._put2stdin_queue.put(cmdstr.encode(outencode, errors='ignore'))
        return cmdstr
    def cmd(self,cmdstr,outencode='UTF-8'):
        self.kobj._logln("cmd received:"+cmdstr)
        if cmdstr.strip()=='stopsrv':
            self.kobj.stop_srvmode()
        return cmdstr
    def retryexeccode(self):
        self.kobj.do_retryexeccode()
        return
    def stopsrv(self):
        self.kobj.stop_srvmode()
        return
    

[MyPythonKernel211735] Info:file h:\Jupyter\Myjupyter-kernel\base\../src/_RPCClass.py created successfully
